### **This aims to classify the exosome staus based on a featureset derrived from the peaks.**

#### **This version performas grid search in parralel so may require better hardware.**

Lets test different spectral cleaning parameters

Import Libraries

In [28]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
from scipy import sparse
from scipy.sparse.linalg import spsolve
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_validate, GroupKFold
from joblib import Parallel, delayed


Read the spectral data

In [29]:
#df = pd.read_csv("../../data/exosomes.raw_spectrum_1.csv")
#df = pd.read_csv("../../data/exosomes.raw_spectrum_400-1800.csv")
df = pd.read_csv("../../data/400-1800_spike_removed.csv")

In [30]:
df

,SpecID,Seq,WaveNumber,Absorbance,SurID,Status
0,201210-1-00,293,400.22778,1765.6628,201210-1,Normal
1,201210-1-00,294,400.91116,1774.7809,201210-1,Normal
2,201210-1-00,295,401.59454,1769.0302,201210-1,Normal
3,201210-1-00,296,402.27789,1756.4220,201210-1,Normal
4,201210-1-00,297,402.96127,1758.8690,201210-1,Normal
...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,1617.3926,210526-3,Hyperglycemia
6239201,210526-3-09,2338,1797.72200,1633.0911,210526-3,Hyperglycemia
6239202,210526-3-09,2339,1798.40550,1633.3076,210526-3,Hyperglycemia
6239203,210526-3-09,2340,1799.08890,1641.8665,210526-3,Hyperglycemia


In [31]:
df['SpecID'].unique()

array(['201210-1-00', '201210-1-01', '201210-1-02', ..., '210526-3-07',
       '210526-3-08', '210526-3-09'], dtype=object)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6239205 entries, 0 to 6239204
Data columns (total 6 columns):
 #   Column      Dtype  
---  ------      -----  
 0   SpecID      object 
 1   Seq         int64  
 2   WaveNumber  float64
 3   Absorbance  float64
 4   SurID       object 
 5   Status      object 
dtypes: float64(2), int64(1), object(3)
memory usage: 285.6+ MB


In [33]:
def prepare_wavelength_df(df, absorbance_col, status_col='Status'):

    # Pivot the DataFrame to get wavelengths as columns and absorbance values
    wavelength_df = df.pivot(index='SpecID', columns='WaveNumber', values=absorbance_col).reset_index()
    wavelength_df.columns.name = None

    # Merge with the statuses based on SpecID
    # Include the SurID to perform GroupKFold CV
    statuses_and_surface = df[['SpecID', 'SurID', status_col]].drop_duplicates()
    wavelength_df = pd.merge(wavelength_df, statuses_and_surface, on='SpecID')

    # Set SpecID as the index
    wavelength_df = wavelength_df.set_index('SpecID')

    return wavelength_df

In [34]:
wavelength_df = prepare_wavelength_df(df, 'Absorbance')

In [35]:
wavelength_df.head()

,400.22778,400.91116,401.59454,402.27789,402.96127,403.64465,404.32803,405.01138,405.69476,406.37814,...,1794.9886,1795.672,1796.3553,1797.0387,1797.722,1798.4055,1799.0889,1799.7722,SurID,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,1765.6628,1774.7809,1769.0302,1756.4220,1758.8690,1763.2300,1745.2285,1773.3534,1774.7166,1753.3281,...,1210.4993,1213.9619,1225.2153,1210.0010,1210.6858,1194.4679,1195.1451,1189.8683,201210-1,Normal
201210-1-01,1966.9930,1962.4237,1954.5616,1954.3228,1963.0917,1975.0807,1979.3162,1963.4561,1968.4587,1964.0000,...,1382.6973,1363.7004,1360.6210,1354.0477,1353.0381,1353.9978,1361.2426,1370.2874,201210-1,Normal
201210-1-02,2182.6694,2149.6565,2146.0227,2159.3459,2167.2910,2160.9861,2145.6575,2134.2004,2142.8303,2138.6309,...,1976.2070,1989.0183,1996.2838,1979.3507,1976.2002,1994.9839,1974.2030,1971.1880,201210-1,Normal
201210-1-03,2445.0837,2430.4973,2422.7927,2434.3433,2454.9700,2462.8245,2454.7007,2467.7329,2449.5161,2421.3474,...,1992.3817,2022.6331,2001.8311,2010.0946,2006.4933,2017.2891,2038.1699,2000.6475,201210-1,Normal
201210-1-04,2250.4536,2248.6235,2245.0984,2242.7173,2235.2803,2228.9585,2236.0095,2229.6091,2225.9231,2211.0359,...,2009.0385,1953.3303,1963.5698,1964.5299,1969.5634,1986.6266,1970.1484,2007.0848,201210-1,Normal


In [36]:
def normalise(absorbances):
    max_value = np.max(absorbances)
    normalised_absorbances = absorbances / max_value
    return normalised_absorbances

Try with vector scaling instead.

In [ ]:
def normalise(absorbances):
    l2_norm = np.sqrt(np.sum(absorbances**2))  # Calculate the euclidean norm
    normalised_absorbances = absorbances / l2_norm
    return normalised_absorbances

>#### **Perform Grid-Search to find the best Assymetric Least Squares Parameters**

Set the Baseline Correction and Smoothing Parameters to Search

In [37]:
## Testing
# lam_values = [10**7, 10**8]
# p_values = [0.001]
# window_size = [9, 101]
# poly_order = [1]

# First Search
lam_values = [10**2, 10**3, 10**4, 10**5, 10**6, 10**7, 10**8, 10**9]
p_values = [0.001, 0.005, 0.01, 0.05, 0.1]
window_size = [5, 7, 9, 19, 25, 35, 45, 51, 65, 75, 85, 95, 101, 151, 201]
poly_order = [1, 2, 3]

In [38]:
df

,SpecID,Seq,WaveNumber,Absorbance,SurID,Status
0,201210-1-00,293,400.22778,1765.6628,201210-1,Normal
1,201210-1-00,294,400.91116,1774.7809,201210-1,Normal
2,201210-1-00,295,401.59454,1769.0302,201210-1,Normal
3,201210-1-00,296,402.27789,1756.4220,201210-1,Normal
4,201210-1-00,297,402.96127,1758.8690,201210-1,Normal
...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,1617.3926,210526-3,Hyperglycemia
6239201,210526-3-09,2338,1797.72200,1633.0911,210526-3,Hyperglycemia
6239202,210526-3-09,2339,1798.40550,1633.3076,210526-3,Hyperglycemia
6239203,210526-3-09,2340,1799.08890,1641.8665,210526-3,Hyperglycemia


In [39]:
df = df.drop(columns=['Seq'])

Pybaselines is actually much faster.

In [40]:
from pybaselines.whittaker import asls

def asls_baseline_correction(x, lam, p):
        corrected, _ = asls(x, lam=lam, p=p)
        return x - corrected

In [41]:
def process_combination(lam, p, df):
    df_copy = df.copy()  # Work on a copy to avoid modifying the original df in-place

    df_copy['Baseline_Corrected_Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: asls_baseline_correction(x, lam, p))
    
    # Assuming prepare_wavelength_df function processes the df_copy and returns a DataFrame ready for ML
    baseline_corrected = prepare_wavelength_df(df_copy, 'Baseline_Corrected_Absorbance')
    X = baseline_corrected.drop(['Status', 'SurID'], axis=1)
    y = baseline_corrected['Status']
    groups = baseline_corrected['SurID']
    
    et = ExtraTreesClassifier(random_state=1234)
    cv = GroupKFold(n_splits=10)
    
    scores = cross_validate(et, X, y, groups=groups, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])
    
    return {
        'lam': lam,
        'p': p,
        'Accuracy': np.mean(scores['test_accuracy']),
        'Precision': np.mean(scores['test_precision_macro']),
        'Recall': np.mean(scores['test_recall_macro']),
        'F1': np.mean(scores['test_f1_macro']),
        'Accuracy Std': np.std(scores['test_accuracy']),
        'Precision Std': np.std(scores['test_precision_macro']),
        'Recall Std': np.std(scores['test_recall_macro']),
        'F1 Std': np.std(scores['test_f1_macro'])
    }

# Parallel execution
results = Parallel(n_jobs=-1)(delayed(process_combination)(lam, p, df) for lam in lam_values for p in p_values)

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Identify the best parameters based on the selected criteria
best_row = results_df.loc[results_df['Accuracy'].idxmax()]

print(f"Best Parameters: lam={best_row['lam']}, p={best_row['p']} with Accuracy: {best_row['Accuracy']:.4f}")

# Save the results to csv
results_df.sort_values('Accuracy', ascending=False).to_csv("Baseline_Corrected_Results.csv", index=False)

Best Parameters: lam=10000000.0, p=0.001 with Accuracy: 0.5798


>#### **Try with Different Smoothing Parameters**

In [ ]:
def process_combination(window, poly, df):
    df_copy = df.copy()  # Working on a copy to avoid modifying the original df
    df_copy['Smoothed_Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: savgol_filter(x, window, poly, deriv=0))
    smoothed_absorbance = prepare_wavelength_df(df_copy, 'Smoothed_Absorbance')

    X = smoothed_absorbance.drop(['Status', 'SurID'], axis=1)
    y = smoothed_absorbance['Status']
    groups = smoothed_absorbance['SurID']
    
    et = ExtraTreesClassifier(random_state=1234)
    cv = GroupKFold(n_splits=10)
    
    scores = cross_validate(et, X, y, groups=groups, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])
    
    return {
        'window': window,
        'poly': poly,
        'Accuracy': np.mean(scores['test_accuracy']),
        'Precision': np.mean(scores['test_precision_macro']),
        'Recall': np.mean(scores['test_recall_macro']),
        'F1': np.mean(scores['test_f1_macro']),
        'Accuracy Std': np.std(scores['test_accuracy']),
        'Precision Std': np.std(scores['test_precision_macro']),
        'Recall Std': np.std(scores['test_recall_macro']),
        'F1 Std': np.std(scores['test_f1_macro'])
    }

# Execute in parallel
results = Parallel(n_jobs=-1)(delayed(process_combination)(window, poly, df) for window in window_size for poly in poly_order)

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Identify the best parameters based on the selected criteria
best_row = results_df.loc[results_df['Accuracy'].idxmax()]

print(f"Best Parameters: window_size={best_row['window']}, poly_order={best_row['poly']} with Accuracy: {best_row['Accuracy']:.4f}")

# Save the results to csv
results_df.sort_values('Accuracy', ascending=False).to_csv("Smoothed_Absorbance_Results.csv", index=False)

Best Parameters: window_size=9.0, poly_order=1.0 with Accuracy: 0.5331


>#### **Try with both**

In [ ]:
def compute_baseline_correction(lam, p, df):

    # Compute the baseline correction for a given lambda and p.

    df_copy = df.copy()
    df_copy['Baseline_Corrected_Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: asls_baseline_correction(x, lam, p))
    
    # Returning both the corrected DataFrame and the lam, p parameters for later use
    return lam, p, df_copy

def evaluate_model(lam, p, window, poly, df_corrected):

    # Apply smoothing and evaluate the model for given lam, p, window, and poly on a baseline-corrected DataFrame.
    
    df_smoothed = df_corrected.copy()
    df_smoothed['Smoothed_Baseline'] = df_smoothed.groupby('SpecID')['Baseline_Corrected_Absorbance'].transform(lambda x: savgol_filter(x, window, poly, deriv=0))
    smoothed_baseline = prepare_wavelength_df(df_smoothed, 'Smoothed_Baseline')

    X = smoothed_baseline.drop(['Status', 'SurID'], axis=1)
    y = smoothed_baseline['Status']
    groups = smoothed_baseline['SurID']
    
    et = ExtraTreesClassifier(random_state=1234)
    cv = GroupKFold(n_splits=10)
    
    scores = cross_validate(et, X, y, groups=groups, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])
    
    return {
        'lam': lam,
        'p': p,
        'window': window,
        'poly': poly,
        'Accuracy': np.mean(scores['test_accuracy']),
        'Precision': np.mean(scores['test_precision_macro']),
        'Recall': np.mean(scores['test_recall_macro']),
        'F1': np.mean(scores['test_f1_macro']),
        'Accuracy Std': np.std(scores['test_accuracy']),
        'Precision Std': np.std(scores['test_precision_macro']),
        'Recall Std': np.std(scores['test_recall_macro']),
        'F1 Std': np.std(scores['test_f1_macro'])
    }

# Step 1: Compute all unique baseline corrections in parallel
baseline_corrections = Parallel(n_jobs=-1)(
    delayed(compute_baseline_correction)(lam, p, df) for lam in lam_values for p in p_values
)

# Step 2: For each baseline-corrected series, perform smoothing and model evaluation in parallel
results = []
for lam, p, df_corrected in baseline_corrections:
    results.extend(
        Parallel(n_jobs=-1)(
            delayed(evaluate_model)(lam, p, window, poly, df_corrected) 
            for window in window_size 
            for poly in poly_order
        )
    )

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Identify the best parameters based on the selected criteria
best_row = results_df.loc[results_df['Accuracy'].idxmax()]

print(f"Best Parameters: lam={best_row['lam']}, p={best_row['p']}, window_size={best_row['window']}, poly_order={best_row['poly']} with Accuracy: {best_row['Accuracy']:.4f}")

# Save the results to csv
results_df.sort_values('Accuracy', ascending=False).to_csv("Smoothed_Baseline_Results.csv", index=False)

Best Parameters: lam=10000000.0, p=0.001, window_size=9.0, poly_order=1.0 with Accuracy: 0.6047


>#### **Finally Try this using again with scaling**

In [ ]:
def process_combination(lam, p, df):
    df_copy = df.copy()  # Work on a copy to avoid modifying the original df in-place
    df_copy['Baseline_Corrected_Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: asls_baseline_correction(x, lam, p))
    df_copy['Scaled_Baseline'] = df_copy.groupby('SpecID')['Baseline_Corrected_Absorbance'].transform(lambda x: normalise(x))

    baseline_corrected = prepare_wavelength_df(df_copy, 'Scaled_Baseline')
    X = baseline_corrected.drop(['Status', 'SurID'], axis=1)
    y = baseline_corrected['Status']
    groups = baseline_corrected['SurID']
    
    et = ExtraTreesClassifier(random_state=1234)
    cv = GroupKFold(n_splits=10)
    
    scores = cross_validate(et, X, y, groups=groups, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])
    
    return {
        'lam': lam,
        'p': p,
        'Accuracy': np.mean(scores['test_accuracy']),
        'Precision': np.mean(scores['test_precision_macro']),
        'Recall': np.mean(scores['test_recall_macro']),
        'F1': np.mean(scores['test_f1_macro']),
        'Accuracy Std': np.std(scores['test_accuracy']),
        'Precision Std': np.std(scores['test_precision_macro']),
        'Recall Std': np.std(scores['test_recall_macro']),
        'F1 Std': np.std(scores['test_f1_macro'])
    }

# Parallel execution
results = Parallel(n_jobs=-1)(delayed(process_combination)(lam, p, df) for lam in lam_values for p in p_values)

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Identify the best parameters based on the selected criteria
best_row = results_df.loc[results_df['Accuracy'].idxmax()]

print(f"Best Parameters: lam={best_row['lam']}, p={best_row['p']} with Accuracy: {best_row['Accuracy']:.4f}")

# Save the results to csv
results_df.sort_values('Accuracy', ascending=False).to_csv("Scaled_Baseline_Corrected_Results.csv", index=False)

Best Parameters: lam=100000000.0, p=0.001 with Accuracy: 0.5444


>#### **Try with Different Smoothing Parameters**

In [ ]:
def process_combination(window, poly, df):
    df_copy = df.copy()  # Working on a copy to avoid modifying the original df
    df_copy['Smoothed_Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: savgol_filter(x, window, poly, deriv=0))
    df_copy['Scaled_Smooth'] = df_copy.groupby('SpecID')['Smoothed_Absorbance'].transform(lambda x: normalise(x))

    smoothed_absorbance = prepare_wavelength_df(df_copy, 'Scaled_Smooth')

    X = smoothed_absorbance.drop(['Status', 'SurID'], axis=1)
    y = smoothed_absorbance['Status']
    groups = smoothed_absorbance['SurID']
    
    et = ExtraTreesClassifier(random_state=1234)
    cv = GroupKFold(n_splits=10)
    
    scores = cross_validate(et, X, y, groups=groups, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])
    
    return {
        'window': window,
        'poly': poly,
        'Accuracy': np.mean(scores['test_accuracy']),
        'Precision': np.mean(scores['test_precision_macro']),
        'Recall': np.mean(scores['test_recall_macro']),
        'F1': np.mean(scores['test_f1_macro']),
        'Accuracy Std': np.std(scores['test_accuracy']),
        'Precision Std': np.std(scores['test_precision_macro']),
        'Recall Std': np.std(scores['test_recall_macro']),
        'F1 Std': np.std(scores['test_f1_macro'])
    }

# Execute in parallel
results = Parallel(n_jobs=-1)(delayed(process_combination)(window, poly, df) for window in window_size for poly in poly_order)

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Identify the best parameters based on the selected criteria
best_row = results_df.loc[results_df['Accuracy'].idxmax()]

print(f"Best Parameters: window_size={best_row['window']}, poly_order={best_row['poly']} with Accuracy: {best_row['Accuracy']:.4f}")

# Save the results to csv
results_df.sort_values('Accuracy', ascending=False).to_csv("Scaled_Smoothed_Absorbance_Results.csv", index=False)

Best Parameters: window_size=9.0, poly_order=1.0 with Accuracy: 0.4968


>#### **Try with both**

In [ ]:
def compute_baseline_correction(lam, p, df):

    # Compute the baseline correction for a given lambda and p.

    df_copy = df.copy()
    df_copy['Baseline_Corrected_Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: asls_baseline_correction(x, lam, p))
    # Returning both the corrected DataFrame and the lam, p parameters for later use
    return lam, p, df_copy

def evaluate_model(lam, p, window, poly, df_corrected):

    # Apply smoothing and evaluate the model for given lam, p, window, and poly on a baseline-corrected DataFrame.
    
    df_smoothed = df_corrected.copy()
    df_smoothed['Smoothed_Baseline'] = df_smoothed.groupby('SpecID')['Baseline_Corrected_Absorbance'].transform(lambda x: savgol_filter(x, window, poly, deriv=0))
    df_smoothed['Scaled_Smooth_Baseline'] = df_smoothed.groupby('SpecID')['Smoothed_Baseline'].transform(lambda x: normalise(x))
    
    smoothed_baseline = prepare_wavelength_df(df_smoothed, 'Scaled_Smooth_Baseline')
    
    X = smoothed_baseline.drop(['Status', 'SurID'], axis=1)
    y = smoothed_baseline['Status']
    groups = smoothed_baseline['SurID']
    
    et = ExtraTreesClassifier(random_state=1234)
    cv = GroupKFold(n_splits=10)
    
    scores = cross_validate(et, X, y, groups=groups, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])
    
    return {
        'lam': lam,
        'p': p,
        'window': window,
        'poly': poly,
        'Accuracy': np.mean(scores['test_accuracy']),
        'Precision': np.mean(scores['test_precision_macro']),
        'Recall': np.mean(scores['test_recall_macro']),
        'F1': np.mean(scores['test_f1_macro']),
        'Accuracy Std': np.std(scores['test_accuracy']),
        'Precision Std': np.std(scores['test_precision_macro']),
        'Recall Std': np.std(scores['test_recall_macro']),
        'F1 Std': np.std(scores['test_f1_macro'])
    }

# Step 1: Compute all unique baseline corrections in parallel
baseline_corrections = Parallel(n_jobs=-1)(
    delayed(compute_baseline_correction)(lam, p, df) for lam in lam_values for p in p_values
)

# Step 2: For each baseline-corrected series, perform smoothing and model evaluation in parallel
results = []
for lam, p, df_corrected in baseline_corrections:
    results.extend(
        Parallel(n_jobs=-1)(
            delayed(evaluate_model)(lam, p, window, poly, df_corrected) 
            for window in window_size 
            for poly in poly_order
        )
    )

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Identify the best parameters based on the selected criteria
best_row = results_df.loc[results_df['Accuracy'].idxmax()]

print(f"Best Parameters: lam={best_row['lam']}, p={best_row['p']}, window_size={best_row['window']}, poly_order={best_row['poly']} with Accuracy: {best_row['Accuracy']:.4f}")

# Save the results to csv
results_df.sort_values('Accuracy', ascending=False).to_csv("Scaled_Smoothed_Baseline_Results.csv", index=False)

Best Parameters: lam=100000000.0, p=0.001, window_size=9.0, poly_order=1.0 with Accuracy: 0.5565
